Imports necessary to run

In [1]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import re
import os

``tmp`` will be the destination directory for the downloaded files. 

In [97]:
cwd = os.getcwd()
try:
    if( not os.path.exists(cwd+'\\tmp')):
        os.mkdir(cwd+'\\tmp')
except Exception as e:
    print(e)

Important ``configs`` for the browser

In [112]:
options = webdriver.ChromeOptions()

options.add_experimental_option("prefs", {
    'download.prompt_for_download':False,
    'plugins.always_open_pdf_externally':True,
    "download.default_directory": cwd+'\\tmp',
    "safebrowsing_for_trusted_sources_enabled": False,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

In [143]:
# base_url = "https://ocw.mit.edu/" 
# base_url = "https://ocw.mit.edu/courses/18-s191-introduction-to-computational-thinking-fall-2022/download/" 
# base_url = "https://ocw.mit.edu/search/?q=database&t=Computer%20Science" 
# base_url = "https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-fall-2010/download/" 
# base_url = "https://ocw.mit.edu/courses/introduction-to-r-and-gis-fall-2023/download/" 
base_url = "https://ocw.mit.edu/courses/11-522-research-seminar-on-urban-information-systems-fall-2005/download/" 
# base_url = "https://ocw.mit.edu/search/?t=Computer%20Science" 

browser = webdriver.Chrome(options=options)  # Optional argument, if not specified will search path.

browser.get(base_url)
browser.implicitly_wait(5)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

``Courses`` will be the directory to structure the scrapped contents

In [100]:
try:
    if(not os.path.exists(cwd+'\\Courses')):
        os.mkdir(cwd+'\\Courses')
except Exception as e:
    print(e)

The ``counter`` will be a pointer which show the current course

In [60]:
counter = 0

This block will enter the first page of the course and will see if it has content to be downloaded

In [139]:
clickable = browser.find_element(By.ID, f'search-result-{counter}-title')
clickable.click()
browser.implicitly_wait(5)

try:
    click_download = browser.find_element(By.XPATH, '//html/body/div[1]/div[1]/div[2]/div[3]/div/div/div/div[2]/div/a')
except NoSuchElementException as e:
    print('Nothing to Download...')
    counter+=1
    browser.execute_script("window.history.go(-1)")

click_download.click()
browser.implicitly_wait(5)

ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [116]:
html = browser.page_source
soup = BeautifulSoup(html)

This block will see the contents to be downloaded, if thats none, then go back to the initial page, if it has then lets start the digging

In [117]:
resources = soup.find_all("div", class_= "resource-list-toggle")
# print(resources)
if len(resources)==0:
    counter+=1
    browser.execute_script("window.history.go(-1)")
    browser.execute_script("window.history.go(-1)")
    pass

title:str = soup.find('h1').text
title = title.strip('/\n')
title = title.replace(':','-')
title = '_'.join(title.split(' '))

try:
    if(not os.path.exists(cwd+'\\Courses'+'\\'+title)):
        os.mkdir(cwd+'\\Courses'+'\\'+title)
except Exception as e:
    print(e)

Troubleshot some complexity in the HTML Structure

In [108]:
checker = soup.find_all("div", class_= "resource-list")
indexes = []

for i,x in enumerate(checker):
    if x.find('h4'):
        indexes.append(i)

In [45]:
# html = browser.page_source
# soup = BeautifulSoup(html)
# res = soup.find_all('a',class_='resource-thumbnail')
# # print(len(res))
# for x in res:
#     click_assignment_download_sell_all = browser.find_element(By.XPATH,f'//a[contains(@href,"{x.attrs['href']}")]')
#     click_assignment_download_sell_all.click()
#     time.sleep(0.5)
# has_see_all=True
# browser.implicitly_wait(2)
# browser.execute_script("window.history.go(-1)")

In [118]:
resources

[<div class="resource-list-toggle">
 <a aria-controls="resource-list-container-lecture-videos" aria-expanded="true" class="py-3" data-toggle="collapse" href="#resource-list-container-lecture-videos">
 <i aria-hidden="true" class="material-icons"></i>
 <h4 class="my-4">Lecture Videos</h4>
 </a>
 </div>,
 <div class="resource-list-toggle">
 <a aria-controls="resource-list-container-programming-assignments" aria-expanded="false" class="collapsed py-3" data-toggle="collapse" href="#resource-list-container-programming-assignments">
 <i aria-hidden="true" class="material-icons"></i>
 <h4 class="my-4">Programming Assignments</h4>
 </a>
 </div>]

In [124]:
for i,x in enumerate(res):
    print(re.search('\..{1,4}$',x.attrs['href']))

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
C:\Users\papal\AppData\Local\Temp\ipykernel_10300\3672465039.py:2: SyntaxWarning: invalid escape sequence '\.'
  print(re.search('\..{1,4}$',x.attrs['href']))
C:\Users\papal\AppData\Local\Temp\ipykernel_10300\3672465039.py:2: SyntaxWarning: invalid escape sequence '\.'
  print(re.search('\..{1,4}$',x.attrs['href']))


KeyError: 'href'

In [136]:
for i,x in enumerate(resources):
    
    html = browser.page_source
    soup = BeautifulSoup(html)
    
    section_name:str = x.find('h4').text
   
    # print(indexes[i]+1)
    # print(f'//*[@id="main-course-section"]/div/div[2]/div[{indexes[i]+1}]/div[1]/a')
   
    if len(resources) == 1:
        click_lectures = browser.find_element(By.XPATH, f'//*[@id="main-course-section"]/div/div[2]/div/div[1]/a')
    else:
        # print(f'//*[@id="main-course-section"]/div/div[2]/div[{indexes[i]+1}]/div[1]/a')
        click_lectures = browser.find_element(By.XPATH, f'//*[@id="main-course-section"]/div/div[2]/div[{indexes[i]+1}]/div[1]/a')

    click_lectures.click()
    time.sleep(1)
    
    has_see_all=False
    try:
        click_see_all = browser.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div/div[1]/div/div[2]/div[{indexes[i]+1}]/div[2]/div[11]/div/div/a')
        click_see_all.click()
        browser.implicitly_wait(3)
        
        
        html = browser.page_source
        soup = BeautifulSoup(html)
        res = soup.find_all('a',class_='resource-thumbnail')
        
        
        for i,x in enumerate(res):
            if re.search('\..{1,4}$',x.attrs['href']):
                if '.mp4' in x.attrs['href']:
                    mp4_name = soup.find_all('a',class_='resource-list-title')
                    mp4_name:str = mp4_name[i].text
                    mp4_name = mp4_name.replace(' ','_')
                    mp4_name = mp4_name.replace(':','_')
                    
                    
                    f = open("./tmp/" +mp4_name+'.txt', "w")
                    f.write(x.attrs['href'])
                    f.close()
                    continue
                click_assignment_download_sell_all = browser.find_element(By.XPATH,f'//a[contains(@href,"{x.attrs['href']}")]')
                click_assignment_download_sell_all.click()
                time.sleep(1)
            
            
        has_see_all=True
        browser.execute_script("window.history.go(-1)")
        browser.implicitly_wait(3)
            
    except NoSuchElementException as e:
        print('No See All encountered!')
        pass
    
    
    if not has_see_all:
        res = soup.find_all(id=re.compile("resource-list-container"))

        counter_ii = 0
        for i,x in enumerate(res[i].find_all('a', href=True)):
            print(x['href'],i)
            if re.search('\..{1,4}$',x['href']):
                if '.mp4' in x['href']:
                    mp4_name = soup.find_all('a',class_='resource-list-title')
                    print(len(mp4_name))
                    mp4_name:str = mp4_name[counter_ii].text
                    mp4_name = mp4_name.replace(' ','_')
                    mp4_name = mp4_name.replace(':','_')
                    
                    
                    f = open("./tmp/" +mp4_name+'.txt', "w")
                    f.write(x.text)
                    f.close()
                    
                    counter_ii+=1
                    continue
                # print(x)
                # print(x['href'])
                # leactures_pdf.append(x['href'])
                click_assignment_download = browser.find_element(By.XPATH,f'//a[contains(@href,"{x['href']}")]')
                # print(f'//a[contains(@href,"{x['href']}")]')
                click_assignment_download.click()
                time.sleep(0.5)
        
    list_of_files = os.listdir(os.getcwd() + '\\tmp')
    section_name = section_name.replace(':','-')
    section_name = '_'.join(section_name.split(' '))
    
    for file in list_of_files:
        # print(os.getcwd() + '\\tmp'+'\\'+file)
        find_hash =re.findall(r'[a-z0-9]{32}_',file)
        new_name = file
        if(len(find_hash) != 0):
            new_name = file.lstrip(find_hash[0])
        print(new_name)
        os.rename(os.getcwd() + '\\tmp'+'\\'+file,  os.getcwd() + '\\tmp'+'\\' + new_name)
        origin_path = os.getcwd() + '\\tmp'+'\\'+new_name
        try:
            os.mkdir(os.getcwd() + '\\Courses'+'\\'+ title+'\\'+ section_name)
        except:
            pass
        destination_path =os.getcwd() + '\\Courses'+'\\'+title+'\\'+ section_name+'\\'+new_name
        os.replace(origin_path,destination_path)
        
    
    click_lectures.click()
    time.sleep(1)
    
counter+=1
browser.execute_script("window.history.go(-1)")
browser.execute_script("window.history.go(-1)")

<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:65: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:65: SyntaxWarning: invalid escape sequence '\.'
C:\Users\papal\AppData\Local\Temp\ipykernel_10300\1513370906.py:33: SyntaxWarning: invalid escape sequence '\.'
  if re.search('\..{1,4}$',x.attrs['href']):
C:\Users\papal\AppData\Local\Temp\ipykernel_10300\1513370906.py:65: SyntaxWarning: invalid escape sequence '\.'
  if re.search('\..{1,4}$',x['href']):


No See All encountered!
/courses/introduction-to-r-and-gis-fall-2023/res1-002-gis-2a-key-concepts_360p_16_9.mp4 0
12
/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2a-key-concepts_mp4/ 1
/courses/introduction-to-r-and-gis-fall-2023/res1-002-gis-2b-vectorial-maps_360p_16_9.mp4 2
12
/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2b-vectorial-maps_mp4/ 3
/courses/introduction-to-r-and-gis-fall-2023/res1-002-gis-2c-projection_360p_16_9.mp4 4
12
/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2c-projection_mp4/ 5
/courses/introduction-to-r-and-gis-fall-2023/res1-002-gis-2d-stack-brick-crop-mask_360p_16_9.mp4 6
12
/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2d-stack-brick-crop-mask_mp4/ 7
/courses/introduction-to-r-and-gis-fall-2023/res1-002-gis-2e-extraction-info-from-maps_360p_16_9.mp4 8
12
/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2e-extraction-info-from-maps_mp4/ 9
/courses/int

In [134]:
mp4_name

[<a class="resource-list-title" href="/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2a-key-concepts_mp4/">Introduction to GIS, Part I: Key Concepts</a>,
 <a class="resource-list-title" href="/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2b-vectorial-maps_mp4/">Introduction to GIS, Part II: Vectorial Maps, Raster Maps, and Time Series</a>,
 <a class="resource-list-title" href="/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2c-projection_mp4/">Introduction to GIS, Part III: Projection</a>,
 <a class="resource-list-title" href="/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2d-stack-brick-crop-mask_mp4/">Introduction to GIS, Part IV: Stack, Brick, Crop, and Mask</a>,
 <a class="resource-list-title" href="/courses/introduction-to-r-and-gis-fall-2023/resources/res1-002-gis-2e-extraction-info-from-maps_mp4/">Introduction to GIS, Part V: Extract Information from Maps using Spatial Data Points</a>,
 <a class=

In [ ]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [66]:
f = open("demofile2.txt", "w")
f.write(str(1))
f.close()

In [62]:
# re.findall('search-result-10-title',browser.page_source)
f = open("demofile2.txt", "r")
print(f.read())

Now the file has more content!


In [ ]:
# clickable = browser.find_element(By.ID, f'search-result-13-title')
# clickable.click()